### PyTorch CustomNet Exercises

Welcome to the PyTorch CustomNet exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

In [2]:
# Import necessary libraries
import os
import requests
from zipfile import ZipFile
from io import BytesIO
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [3]:
# Define the path to the dataset
dataset_path = 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'  # Replace with the path to your dataset

# Send a GET request to the URL
response = requests.get(dataset_path)
# Check if the request was successful
if response.status_code == 200:
    # Open the downloaded bytes and extract them
    with ZipFile(BytesIO(response.content)) as zip_file:
        zip_file.extractall('/dataset')
    print('Download and extraction complete!')

Download and extraction complete!


In [6]:
# Define the custom neural network
class CustomNet(nn.Module):
    def __init__(self):
        super(CustomNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 200)
        self.pool = nn.MaxPool2d(2, 2)


    def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = F.dropout(x, p=0.5)
      x = self.pool(F.relu(self.conv2(x)))
      x = torch.flatten(x, 1)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

# Define transforms
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to fit the input dimensions of the network
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Load TinyImageNet
train_dataset = datasets.ImageFolder(root='/dataset/tiny-imagenet-200/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/dataset/tiny-imagenet-200/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize the model
model = CustomNet().cuda()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

# Training loop
def train(epoch, model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        outputs = model(inputs)
        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total
    print(f'Train Epoch: {epoch} Loss: {train_loss:.6f} Acc: {train_accuracy:.2f}%')

# Test loop
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    test_loss = test_loss / len(test_loader)
    test_accuracy = 100. * correct / total
    print(f'Test Loss: {test_loss:.6f} Acc: {test_accuracy:.2f}%')
    return test_accuracy

# Run the training and testing
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train(epoch, model, train_loader, criterion, optimizer)
test_accuracy = test(model, test_loader, criterion)

Train Epoch: 1 Loss: 5.266485 Acc: 0.94%
Train Epoch: 2 Loss: 4.996660 Acc: 2.70%
Train Epoch: 3 Loss: 4.826947 Acc: 4.67%
Train Epoch: 4 Loss: 4.640512 Acc: 6.73%
Train Epoch: 5 Loss: 4.496919 Acc: 8.36%
Train Epoch: 6 Loss: 4.383195 Acc: 9.86%
Train Epoch: 7 Loss: 4.276799 Acc: 11.37%
Train Epoch: 8 Loss: 4.176665 Acc: 12.61%
Train Epoch: 9 Loss: 4.083001 Acc: 13.97%
Train Epoch: 10 Loss: 3.985425 Acc: 15.39%
Test Loss: 8.233466 Acc: 0.63%
